In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import illustris_python as il
import physics

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential", "Velocities"]
        }
indices = [3605, 3760]

#indices[0] = 1893

#[544, 953, 1285, 1662]
#[1893, 2164, 3043, 3229, 3403, 3605, 3760] 
#np.genfromtxt("../data/tng-100-3/cutdata/central_ids.txt")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists = [gas, dm, stars]
group_cat = pd.DataFrame({"id": indices})
#Start timer
start = timer()


In [2]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"]) #Add dm masses

subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos", "SubhaloVel", "SubhaloSpin"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [3]:
print("Calculating relative positions and radius")
particle_lists, group_cat = physics.properties.relative_pos_radius(particle_lists, N, group_cat)
group_cat = physics.properties.galaxy_radius(stars, N, group_cat)
group_cat = physics.properties.galaxy_stellar_mass(stars, N, group_cat)
print("Calculating particle masses")
particle_lists, group_cat = physics.properties.total_mass(particle_lists, N, group_cat)
print("Calculating subhalo velocity")
particle_lists, group_cat = physics.properties.subhalo_velocity(particle_lists, N, group_cat)
print("Calculating relative velocities")
for particle in particle_lists:
    particle = physics.properties.relative_velocities(particle, N, group_cat)

print("Calculating half mass radius")
stars, group_cat = physics.properties.half_mass_radius(stars, N, group_cat, "Stellar")
"""
print("Calculating maximum angular momentum")
group_cat = physics.properties.max_ang_momentum(stars, N, group_cat)
rot_vector = np.transpose(np.array([group_cat["RotationAxisX"], group_cat["RotationAxisY"], group_cat["RotationAxisZ"]]))
print("Rotating subhalos")
particle_lists = physics.geometry.rotate_coordinates(particle_lists, N, group_cat, rot_vector)
"""
#End timer
end = timer()

Calculating relative positions and radius
Calculating particle masses
Calculating subhalo velocity
Calculating relative velocities
Calculating half mass radius


In [7]:
print("Masses check")
for i in range(2):
    print("Group catalogue: ", df_subhalos["SubhaloMassStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloMassStellar"][i])
    print("Galaxy: ", group_cat["SubhaloGalaxyMassStellar"][i])

Masses check
Group catalogue:  16.47061  Snapshot:  16.4705867767334
Galaxy:  14.0736083984375
Group catalogue:  38.428806  Snapshot:  38.428741455078125
Galaxy:  31.646684646606445


In [7]:
print("Velocities check")
for i in range(3):
    print(group_cat["SubhaloVelX"][i], group_cat["SubhaloVelY"][i], group_cat["SubhaloVelZ"][i])
    print(df_subhalos["SubhaloVel"][indices[i]])

Velocities check
-175.11496411068748 81.26290417921925 -8.336148174927477
[-175.13785    81.367256   -8.339443]
279.265925652316 -99.84715851410176 82.07070970468858
[279.23257  -99.84879   82.072495]
174.67770423321207 57.30106101855136 170.3806483246425
[174.64732  57.31744 170.41402]


In [8]:
print("Halfmass radius check")
for i in range(3): 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][i])

Halfmass radius check
Group catalogue:  64.70497  Snapshot:  64.7262886582862
Group catalogue:  79.047134  Snapshot:  78.95183316992888
Group catalogue:  33.925236  Snapshot:  33.92006088473392


In [9]:

#Save all fields for particles in one subhalo for later inspection
#g = 0
#stars[g].to_pickle("../data/tng-100-3/subhalos/subhalo" + str(indices[g]) +"_stars.pkl")

In [10]:
#Save group catalogue
#group_cat.to_pickle("../data/tng-100-3/catalogues/test_cat_01.pkl")

In [11]:

print("Time to process " + str(N) + " Subhalos: ")
print(int(end - start), "Seconds")

Time to process 4 Subhalos: 
37 Seconds
